In [1]:
import pandas as pd
import re
import numpy as np
import csv
import spacy

In [92]:
file = pd.read_csv('all.csv')
file[0:3]

,text,link,name,description
0,Grr. Why can't people just tell you how they f...,https://en.wikiquote.org/wiki/George_R._R._Martin,George R. R. Martin - Wikiquotehttps://en.wiki...,"I think that for science fiction, fantasy, and..."
1,Grr. Why can't people just tell you how they f...,https://www.rollingstone.com/culture/culture-n...,'Game of Thrones' Author George R.R. Martin: T...,23 Apr 2014 - I don't think that he ever under...
2,Grr. Why can't people just tell you how they f...,NaN,The Teen Report - Apr 2001 - Page 34 - Google ...,"Our source: Isabel Gonzalez, the trendspotting..."


In [70]:
print(len(file))

18315


preprocess data: the preprocess here is just lower case 

In [71]:
#tokenize
def preprocess(sent):

    words = str(sent).lower().split()
    new_words = []
    for w in words:
        w = re.sub(r'[0-9]+', '', w)
        new_words.append(w)
        
    return ' '.join(new_words)


1. Here we create a class to store each variable as an object 
2. Return the count of key words 'lyric, lyrics, quote, quotes, song, songs' in the name of the website, because the name of the website contains most of the information we need
3. Count the keywords and store it as Score. Score is a vector that contain keyword counts in each search result

In [95]:
class myQuote:
    def __init__(self, text):
        #read in CSV, process
        self.quoteText = text
        self.quoteID = hash(self.quoteText)
        self.link = []
        self.name = []
        self.description = []
        self.scores = []
    
    def __hash__(self):
        return self.quoteID
    
    def __str__(self):
        return "Object text: " + self.quoteText + '\n' +\
        "Links: " + str(self.link) + '\n' +\
        "Names: " + str(self.name) + '\n' +\
        "Description " + str(self.description) + '\n' +\
        "Scores: " + str(self.scores)



objects = {}
with open('all.csv', 'r') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        #print(row['text'])
        texthash = hash(row['text'])
        if texthash not in objects:
            objects[texthash] = myQuote(row['text'])
        objects[texthash].link.append(row['link'])
        objects[texthash].name.append(row['name'])
        objects[texthash].description.append(row['description'])
        line = preprocess(row['name'])
        count = line.count("lyric") + line.count("lyrics") + line.count("quote") + line.count("quotes")
        objects[texthash].scores.append(count)
        #add extra fields in the class if I need to add more fields        
#print(len(objects.keys()))
#for item in objects.keys():
#    if sum(objects[item].scores) > 0:
#        print(objects[item])
        
def saveAsCSV(dictz,filename):
    f = open(filename, 'w')
    writer = csv.writer(f, delimiter = ',',quoting=csv.QUOTE_MINIMAL)  
    writer.writerow(["hash"] + ["text"] + ["count"])
    for item in objects.keys():
        writer.writerow([objects[item].quoteID] + [objects[item].quoteText] + [objects[item].scores])
    f.close()
    
saveAsCSV(objects, 'myprocessedquotes2.csv')
    

select and return those has at least search results contain at least one keyword and save it as csv

In [96]:
def SearchResult1(results, filename):
    f = open(filename, 'w')
    writer = csv.writer(f, delimiter = ',',quoting=csv.QUOTE_MINIMAL)  
    writer.writerow(["hash"] + ["text"] + ["count"]+ ["name"])
    for item in results:
        count = 0
        for score in objects[item].scores:
            if score > 0:
                count = count + 1
        if count > 2:
            writer.writerow([objects[item].quoteID] + [objects[item].quoteText] + [objects[item].scores] + [objects[item].description] )
    f.close()

results = objects.keys()
SearchResult1(results, 'QuotesDetected.csv')

In [97]:
#remove punctionations
def preprocess2(sent):
    #remove punctustion
    sent = re.sub(r'[^\w\s]','',sent)
    words = sent.split()
    new_words = []
    for w in words:      
        new_words.append(w.lower())
        
    return ' '.join(new_words)

s = 'Well, here are some of the darkest jokes, found in the deep, disturbing...'
preprocess2(s)

'well here are some of the darkest jokes found in the deep disturbing'

1. select those entries has at least search results (link names) contain more than one keyword, return the score vectors of these entries (entry A). Unselected entries are labeled as NotQuote
2. Among entry (A), compute cosine similarity between posts and content (description) of the search result, those with 90% above similarity are labeled as quote. Those less than 90% are labeled as suspect. 

In [99]:
# use cosine similarity to compare text with search result
nlp = spacy.load('en')
def SearchResult2(results, filename):
    f = open(filename, 'w')
    writer = csv.writer(f, delimiter = ',',quoting=csv.QUOTE_MINIMAL)  
    writer.writerow(["hash"] + ["text"] + ["count"]+ ["cosineSim"] +['label'])
    for item in results:
        count = 0
        for score in objects[item].scores:
            if score > 0:
                count = count + 1
        if count > 2:
            str1 = ''.join (objects[item].description)
            doc2 = nlp(preprocess2(str1))
            doc1 = nlp(preprocess2(objects[item].quoteText))
            if doc1.similarity(doc2) > 0.92:
                writer.writerow([objects[item].quoteID] + [objects[item].quoteText] + [objects[item].scores] + [doc1.similarity(doc2)]+['quote'])
            else:
                writer.writerow([objects[item].quoteID] + [objects[item].quoteText] + [objects[item].scores] + [doc1.similarity(doc2)]+['suspect'])
        else:
            writer.writerow([objects[item].quoteID] + [objects[item].quoteText] + ['null'] + ['null']+['NotQuote'])
    f.close()
        
results = objects.keys()
SearchResult2(results, 'QuotesDetected_all.csv')